Using the functions and loops created in the webscraper notebook.  Here I'm going to loop through one webpage and get all 20 tables into one dataframe

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException
from selectorlib import Extractor
import bs4
import requests
import json
import time
import urllib.request
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd
import numpy as np
from datetime import date

In [2]:
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.add_argument('ignore-certificate-errors')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=ChromeOptions)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/ericroberts/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


In [3]:
url = "https://www.transfermarkt.co.uk/premier-league/sommertransfers/wettbewerb/GB1/saison_id/2020"

driver.get(url)
html = driver.execute_script('return document.body.innerHTML;')
soup = bs4.BeautifulSoup(html,'lxml')

In [4]:
import py_files.transfer_webscraper as transfer_webscraper

# This is how you call it:
# transfer_webscraper.transfer_value_gather(table)

In [5]:
#All the transfer information (stored as 40 tables, 1 for in's and 1 for out's at each club)
#are stored under the tag class_="responsive-table"
#this returns a list of length 40.  The function below extracts the appropriate information.
response_table = soup.find_all(class_="responsive-table")

In [6]:
#This removes the sold players table and makes this list equal in length to the names of the clubs.
list_of_table_of_purchases = []
response_table = soup.find_all(class_="responsive-table")
for i in range(len(response_table)):
    if i % 2 == 0:
        list_of_table_of_purchases.append(response_table[i])

In [7]:
#This creates a list of the clubs in the league in that season in alphabetical order; mirrors the order that 
#response_table is ordered.  Use this along with above function when looking through response_tables to add
#the purchasing club as a column to the df.
list_of_clubs = []
table_header = soup.find_all(class_='table-header')
i=2
while i < 22:
    club = table_header[i].find_all(class_="vereinprofil_tooltip tooltipstered")
    list_of_clubs.append(club[1].text)
    i += 1
list_of_clubs

['Arsenal FC',
 'Aston Villa',
 'Brighton & Hove Albion',
 'Burnley FC',
 'Chelsea FC',
 'Crystal Palace',
 'Everton FC',
 'Fulham FC',
 'Leeds United',
 'Leicester City',
 'Liverpool FC',
 'Manchester City',
 'Manchester United',
 'Newcastle United',
 'Sheffield United',
 'Southampton FC',
 'Tottenham Hotspur',
 'West Bromwich Albion',
 'West Ham United',
 'Wolverhampton Wanderers']

In [15]:
#This loops over the list of clubs (could swap in list of table of purchases - made both lists len 20)
#and runs the scraper function over each table, adds the buying club and then appends it to a df.

Prem_summer_2020 = pd.DataFrame()
for n in range(len(list_of_clubs)):
    df = transfer_webscraper.transfer_value_gather(list_of_table_of_purchases[n])
    df['buying_club'] = list_of_clubs[n]
    Prem_summer_2020 = Prem_summer_2020.append(df)

In [19]:
Prem_summer_2020.tail(50)

,player,age,nationality,position,selling_club,previous_league,est_market_value,fee,buying_club
17,Maya Yoshida,32,Japan,Centre-Back,UC Sampdoria,Italy,£3.42m,"End of loanAug 31, 2020",Southampton FC
0,Giovani Lo Celso,24,Argentina,Attacking Midfield,Real Betis Balompié,Spain,£36.00m,£28.80m,Tottenham Hotspur
1,Sergio Reguilón,23,Spain,Left-Back,Real Madrid,Spain,£22.50m,£27.00m,Tottenham Hotspur
2,Matt Doherty,28,Ireland,Right-Back,Wolverhampton Wanderers,England,£14.40m,£15.12m,Tottenham Hotspur
3,Pierre-Emile Höjbjerg,25,Denmark,Defensive Midfield,Southampton FC,England,£18.00m,£14.94m,Tottenham Hotspur
4,Joe Rodon,22,Wales,Centre-Back,Swansea City,England,£6.30m,£10.89m,Tottenham Hotspur
5,Carlos Vinícius,25,Brazil,Centre-Forward,SL Benfica,Portugal,£18.00m,Loan fee:£2.70m,Tottenham Hotspur
6,Joe Hart,33,England,Goalkeeper,Burnley FC,England,£2.16m,free transfer,Tottenham Hotspur
7,Gareth Bale,31,Wales,Right Winger,Real Madrid,Spain,£25.20m,loan transfer,Tottenham Hotspur
8,Oliver Skipp,20,England,Defensive Midfield,Norwich City,England,£6.48m,"End of loanMay 31, 2021",Tottenham Hotspur


In [ ]:
#From the transfermarkt website creating these lists for the url's and to add variables:
leagues = []

https://www.transfermarkt.co.uk/premier-league/sommertransfers/wettbewerb/GB1/saison_id/2020
https://www.transfermarkt.co.uk/ligue-1/sommertransfers/wettbewerb/FR1
https://www.transfermarkt.co.uk/primera-division/sommertransfers/wettbewerb/ES1
https://www.transfermarkt.co.uk/1-bundesliga/sommertransfers/wettbewerb/L1
https://www.transfermarkt.co.uk/serie-a/sommertransfers/wettbewerb/IT1
https://www.transfermarkt.co.uk/primeira-liga/sommertransfers/wettbewerb/PO1
https://www.transfermarkt.co.uk/premier-liga/sommertransfers/wettbewerb/RU1
    
    
https://www.transfermarkt.co.uk/eredivisie/sommertransfers/wettbewerb/NL1    
    https://www.transfermarkt.co.uk/ligue-1/sommertransfers/wettbewerb/FR1/plus/?saison_id=2020&s_w=s&leihe=1&intern=0&intern=1

    https://www.transfermarkt.co.uk/laliga/sommertransfers/wettbewerb/ES1/plus/?saison_id=2020&s_w=s&leihe=1&intern=0&intern=1